In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import json
import dotenv

In [3]:
dotenv.load_dotenv()

True

In [4]:
from typing import Literal, Annotated
from typing_extensions import TypedDict
from IPython.display import display, Markdown

In [5]:

from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

In [6]:

from langgraph.graph import END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_experimental.utilities import PythonREPL
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START

In [7]:
from src.utils_stock import get_top_nasdaq_stock_data
from src.utils_news import search_and_scrape_url
from src.utils_email import send_report_via_email

In [8]:
# parameter
NUM_DAYS = 10
EMAIL_ADDRESS = "nelsonlin0321@gmail.com"

In [9]:
def get_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        "Use the provided tools to progress towards answering the question."
        f"\n{suffix}"
    )

In [10]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

In [11]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    number_of_days_analysis: int
    email_address: str

### Data Acquisition Agent

In [12]:
@tool
def get_top_nasdaq_performance_stock_data(
) -> str:
    """

    Get the day's top NASDAQ-100 gainer with symbol, percentage increase, sample data in markdown format, pandas dataframe info and csv data path for further python code analysis.

    Returns:
        str: Top nasdaq performance stock data information
    """

    result = get_top_nasdaq_stock_data(NUM_DAYS)
    return json.dumps(result)

In [13]:
data_acquisition_agent = create_react_agent(
    llm,
    tools=[get_top_nasdaq_performance_stock_data],
    prompt=get_system_prompt(
        "You are a data acquisition specialist focused on retrieving NASDAQ market data. Your task is to fetch information about today's top performing NASDAQ stock using the get_top_nasdaq_performance_stock_data tool. The data should include the stock symbol, percentage gain, and relevant trading data for analysis."
    ),
)

### Python Data Analysis Agent

In [14]:
repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to do comprehensive stock performance analysis"],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        print(f"Executing codes:{code}")
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [15]:
python_data_analysis_agent = create_react_agent(
    llm,
    [python_repl_tool],
    prompt=get_system_prompt(
        """
        You are an agent designed to write and execute python code for comprehensive stock analysis given data. 
        The data include the stock symbol, percentage increase, sample data in markdown format, pandas dataframe info, and csv data path for python code analysis. 
        You have access to a python REPL, which you can use to execute python code.
        If you get an error, debug your code and try again.
        

        Your tasks are to generate codes and execute codes with python REPL to fullfil below all requirements in a single python script. 
        Please call python REPL tool multiple times when necessarily to complete all below tasks.

        If complete all below tasks, please prefix your response with FINAL ANSWER so the we knows to stop.

        1. Data Loading and Preparation:
           - Read and clean the provided CSV data using pandas
           - Handle any missing values appropriately
           - Ensure proper data types for calculations

        2. Technical Analysis:
           - Calculate and analyze moving averages (5-day, 10-day)
           - Compute daily returns and cumulative returns
           - Calculate trading volume trends
           - Determine price momentum indicators
           - Analyze last n trading days performance (average change, trend direction)
           - Calculate volatility metrics (standard deviation, beta)

        3. Statistical Insights:
           - Perform descriptive statistics on price and volume data
           - Calculate risk metrics (Sharpe ratio if applicable)
           - Identify any statistically significant patterns
           - Analyze distribution of returns

        All the result should be using python print() to present in a clear, structured format using print statements so that is visible to the user in the console.
"""
    ),
)

### News Research Agent

In [16]:
@tool
def get_stock_news(query: Annotated[str, "The query to get the stock financial news"]) -> str:
    """Use this to do google search news to perform stock news analysis. The results tool will return the news_url with content for your analysis"""
    search_result = search_and_scrape_url(query)
    return json.dumps(search_result)

In [17]:
news_research_agent = create_react_agent(
    llm,
    [get_stock_news],
    prompt=get_system_prompt(
        """
        You are an agent designed to do comprehensive stock news research using google news search.
        Your tasks include:
        1. Sentiment Analysis:
           - Analyze the overall market sentiment (bullish/bearish)
           - Identify key positive and negative factors affecting the stock
           - Evaluate the tone of recent news coverage
        
        2. Risk Assessment:
           - Identify potential risks and challenges mentioned in news
           - Analyze regulatory or legal concerns
           - Evaluate market competition and industry dynamics
        
        3. News Impact Analysis:
           - Assess how recent news might affect stock price
           - Look for patterns in news coverage
           - Identify significant company events or announcements
        
        4. Market Context:
           - Consider broader market conditions
           - Analyze sector-specific news and trends
           - Compare with competitor news and performance
        
        Provide clear, structured insights based on the news data to support investment decisions.
        """
    ),
)

### Reporting Agent

In [18]:
# @tool
# def send_report(
#         subject: Annotated[str, "Email subject of report for email sending"],
#         body: Annotated[str, "Report content in markdown from reporting agent for email sending"]) -> str:
#     """
#     Use this tool to send the generated report.

#     Args:
#         subject (Annotated[str, &quot;Email subject of report for email sending&quot;]): _description_
#         body (Annotated[str, &quot;Report content in markdown from reporting agent for email sending&quot;]):

#     Returns:
#         str: description
#     """
#     result = send_report_via_email(subject, body, EMAIL_ADDRESS)
#     return result

In [19]:
reporting_agent = create_react_agent(
    llm,
    #  tools=[send_report],
    tools=[],
    prompt=get_system_prompt(
        """
        You are a professional financial analyst tasked with synthesizing data and creating comprehensive stock analysis reports.
        
        Your role is to:
        1. You must analyze and integrate findings from two sources:
           - Technical analysis from the python_data_analysis_agent
           - News and sentiment analysis from the news_research_agent
        
        2. Structure your report in two clear sections:
           - Technical Analysis: Interpret and explain the quantitative data findings
           - Market Research & Sentiment: Present key insights from news analysis
        
        3. For each section:
           - Highlight the most significant findings
           - Explain their implications for the stock
           - Support conclusions with specific data points
        
        4. Conclude with:
           - Key takeaways that combine both technical and news analysis
           - Notable risks or opportunities identified
        
        Write in a clear, professional style. Focus on actionable insights.
        Present the report directly without any introductory text or meta-commentary.
        """
    ),
)

### Define Graph

In [20]:
def data_acquisition_agent_node(
    state: State,
) -> Command[Literal["python_data_analysis_agent"]]:   # Go to python_data_analyst node
    result = data_acquisition_agent.invoke(state)
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="data_acquisition_agent"
    )
    return Command(
        update={
            **state,
            "messages": result["messages"],
        },
        goto="python_data_analysis_agent",
    )

In [21]:
def get_next_node_for_data_analysis_agent(last_message: BaseMessage, goto: str):
    if "FINAL ANSWER" in last_message.content:
        return "news_research_agent"
    return goto

In [22]:
def python_data_analysis_agent_node(
    state: State,
) -> Command[Literal["news_research_agent"]]:
    result = python_data_analysis_agent.invoke(state)
    goto = get_next_node_for_data_analysis_agent(
        result["messages"][-1], "python_data_analysis_agent")

    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="python_data_analysis_agent"
    )
    return Command(
        update={
            **state,
            "messages": result["messages"],
        },
        goto=goto,
    )

In [23]:
def news_research_agent_node(
    state: State,
) -> Command[Literal["reporting_agent"]]:  # type: ignore
    result = news_research_agent.invoke(state)
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="news_research_agent"
    )
    return Command(
        update={
            **state,
            "messages": result["messages"],
        },
        goto="reporting_agent",
    )

In [24]:
def reporting_agent_node(
    state: State,
) -> Command[Literal[END]]:  # type: ignore
    result = reporting_agent.invoke(state)
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="reporting_agent"
    )
    return Command(
        update={
            **state,
            "messages": result["messages"],
        },
        goto=END,
    )

In [25]:
workflow = StateGraph(State)
workflow.add_node("data_acquisition_agent", data_acquisition_agent_node)
workflow.add_node("python_data_analysis_agent",
                  python_data_analysis_agent_node)
workflow.add_node("news_research_agent", news_research_agent_node)
workflow.add_node("reporting_agent", reporting_agent_node)

workflow.add_edge(START, "data_acquisition_agent")
graph = workflow.compile()

In [26]:
graph.get_graph().print_ascii()

        +-----------+          
        | __start__ |          
        +-----------+          
               *               
               *               
               *               
  +------------------------+   
  | data_acquisition_agent |   
  +------------------------+   
               .               
               .               
               .               
+----------------------------+ 
| python_data_analysis_agent | 
+----------------------------+ 
               .               
               .               
               .               
    +---------------------+    
    | news_research_agent |    
    +---------------------+    
               .               
               .               
               .               
      +-----------------+      
      | reporting_agent |      
      +-----------------+      
               .               
               .               
               .               
          +---------+          
        

In [27]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	data_acquisition_agent(data_acquisition_agent)
	python_data_analysis_agent(python_data_analysis_agent)
	news_research_agent(news_research_agent)
	reporting_agent(reporting_agent)
	__end__([<p>__end__</p>]):::last
	__start__ --> data_acquisition_agent;
	data_acquisition_agent -.-> python_data_analysis_agent;
	news_research_agent -.-> reporting_agent;
	python_data_analysis_agent -.-> news_research_agent;
	reporting_agent -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc




<img src='https://www.mermaidchart.com/raw/6a579ff6-aa7e-48b7-ae0e-264de058370b?theme=light&version=v0.1&format=svg' width=800, height=600></img>

### Run Graph

In [28]:
input_message = HumanMessage(
    content="First, Let get the top performance stock data")

In [29]:
events = graph.stream(
    input={
        "messages": [input_message],
        "number_of_days_analysis": NUM_DAYS,
        "email_address": EMAIL_ADDRESS
    },
    # Maximum number of steps to take in the graph
    config={"recursion_limit": 15},
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        message = event["messages"][-1]
        message.pretty_print()

================================ Human Message =================================

First, Let get the top performance stock data


100%|██████████| 101/101 [00:02<00:00, 39.25it/s]


================================ Human Message =================================
Name: data_acquisition_agent

Here is the data for today's top-performing NASDAQ stock:

### Stock Details:
- **Symbol**: MNST
- **Percentage Increased**: 3.72%

### Sample Trading Data (Past 5 Days):
| Date                      |   Open |   High |   Low |   Close |   Volume |   Dividends |   Stock Splits |
|:--------------------------|-------:|-------:|------:|--------:|---------:|------------:|---------------:|
| 2025-04-28 00:00:00-04:00 |  58.74 |  58.98 | 58.03 |   58.49 |  3057100 |           0 |              0 |
| 2025-04-29 00:00:00-04:00 |  58.59 |  59.33 | 58.01 |   59.25 |  3448100 |           0 |              0 |
| 2025-04-30 00:00:00-04:00 |  59.59 |  60.29 | 58.8  |   60.12 |  5228700 |           0 |              0 |
| 2025-05-01 00:00:00-04:00 |  59.86 |  60.03 | 59.3  |   59.52 |  5164900 |           0 |              0 |
| 2025-05-02 00:00:00-04:00 |  59.87 |  60.26 | 59.43 |   60.05 |  416

Python REPL can execute arbitrary code. Use with caution.


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2025-04-28 00:00:00-04:00,58.740002,58.980000,58.029999,58.490002,3057100,0.0,0.0
1,2025-04-29 00:00:00-04:00,58.590000,59.330002,58.009998,59.250000,3448100,0.0,0.0
2,2025-04-30 00:00:00-04:00,59.590000,60.290001,58.799999,60.119999,5228700,0.0,0.0
3,2025-05-01 00:00:00-04:00,59.860001,60.029999,59.299999,59.520000,5164900,0.0,0.0
4,2025-05-02 00:00:00-04:00,59.869999,60.259998,59.430000,60.049999,4161600,0.0,0.0


Executing codes:import pandas as pd
import numpy as np

# Load the data
data_path = "/Volumes/mnt/Workspace/stock-analysis-agent-langgraph/data/mnst_performance_in_the_past_10_days.csv"
df = pd.read_csv(data_path)

# Display the first few rows to verify
display(df.head())
Executing codes:# Data Loading and Preparation
print("=== Data Loading and Preparation ===")
print("1. Data Overview:")
print(df.info())
print("\n2. Missing Values:")
print(df.isnull().sum())
print("\n3. Data Types:")
print(df.dtypes)

# Ensure proper data types (if needed)
df['Date'] = pd.to_datetime(df['Date'])
print("\n4. Updated Data Types:")
print(df.dtypes)
Executing codes:# Technical Analysis
print("\n=== Technical Analysis ===")

# Calculate moving averages
df['MA_5'] = df['Close'].rolling(window=5).mean()
df['MA_10'] = df['Close'].rolling(window=10).mean()
print("1. Moving Averages (5-day and 10-day):")
print(df[['Date', 'Close', 'MA_5', 'MA_10']].tail())

# Compute daily returns and cumulative returns
df['Da

# Send Email

In [30]:
body = message.content

title = llm.invoke(
    f"Extract the subject from this report content as the email subject and return the title directly without any introductory text: {body}")

In [31]:
subject = title.content

In [32]:
result = send_report_via_email(subject, body, EMAIL_ADDRESS)
print(result)

Email Sent successfully to nelsonlin0321@gmail.com


In [33]:
display(Markdown(message.content))

### Comprehensive Stock Analysis Report: MNST

---

#### **Technical Analysis**

**Key Findings:**
1. **Price Trends & Moving Averages**  
   - Strong upward momentum: Closing price increased from **58.49 to 61.00** (+4.29%) over 10 days.  
   - 5-day MA rose consistently (59.97 → 60.53), confirming bullish sentiment.  
   - *Implication*: Short-term trend favors buyers, but watch for overbought conditions near the peak (61.00).

2. **Volume & Liquidity**  
   - Volume surged **183%** (3.06M to 8.65M shares), with 5-day average volume at **5.7M**.  
   - *Implication*: High trading activity supports price gains, suggesting strong investor interest.

3. **Risk/Reward Metrics**  
   - **Sharpe Ratio (6.38)**: Exceptional risk-adjusted returns.  
   - **Volatility (σ=0.76)**: Moderate fluctuations—suitable for medium-risk traders.  
   - *Implication*: Attractive for momentum strategies but monitor skewness (-0.84) for downside risks.

---

#### **Market Research & Sentiment**

**Key Insights:**  
*(Note: News data not fetched—hypothetical analysis based on technicals)*  
1. **Potential Catalysts**  
   - Volume spikes often correlate with earnings reports or product launches.  
   - *Action*: Verify if recent news (e.g., new energy drink line) drove the 3.72% surge.

2. **Sector Context**  
   - Beverage sector typically sees seasonal demand shifts.  
   - *Action*: Compare MNST’s performance against peers (e.g., PEP, KO) for relative strength.

3. **Sentiment Indicators**  
   - Left-skewed returns suggest caution among some investors.  
   - *Action*: Check for negative headlines (e.g., regulatory concerns, supply chain issues).

---

#### **Key Takeaways**  
1. **Opportunities**  
   - Strong short-term uptrend supported by volume.  
   - High Sharpe ratio favors tactical long positions.  

2. **Risks**  
   - Overbought signals (peak price + high volume) may precede consolidation.  
   - Negative skewness hints at potential pullbacks.  

3. **Actionable Steps**  
   - **Bullish Scenario**: Enter with tight stop-loss below 5-day MA (60.53).  
   - **Bearish Scenario**: Await RSI data (if extended) or news-driven corrections.  

---

**Final Note**: Pair this analysis with real-time news to validate sentiment. Let me know if you’d like a deeper dive into correlations or visualizations.